# 데이터 분석

##1. 데이터 읽어오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Lec4_BigData/3. 파이프라인/datatitanic_cleaning.csv")
df

,Unnamed: 0,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C
2,2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S
3,3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S
4,4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,886,887,0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,NaN,S
887,887,888,1,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,B42,S
888,888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.699118,1,2,W./C. 6607,23.4500,NaN,S
889,889,890,1,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.0000,C148,C


In [3]:
df.isna().sum()

Unnamed: 0       0
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [4]:
df.drop('Unnamed: 0', axis=1, inplace=True)

>쓸데 없이 있었던 Unnamed뭐시기 삭제

##2. 데이터 구간화 (data binning)
> 연령대 별로 생존 유무를 파악하려 한다. 다음과 같이 구간으로 분리해 보자
* 미성년: 1~20
* 성년: 21~60
* 노년: 61~100

In [5]:
df['Age_group'] = pd.cut(df['Age'], bins=[1, 20 ,60, 100], labels=['미성년', '성년', '노년'])
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_group
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S,성년
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,성년
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S,성년
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,성년
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S,성년
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,NaN,S,성년
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,B42,S,미성년
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.699118,1,2,W./C. 6607,23.4500,NaN,S,성년
889,890,1,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.0000,C148,C,성년


> 데이터를 범위로 구분하기 위해 cut함수를 이ㅛㅇㅇ했다. cut함수에서 bins는 별도로 구간을 나눌 수 있으며, labels는 구간에 별칭을 지정할 수 있다.

> 위의 코드에서 3개의 구간을 나누게 됨으로 age_bins를 1, 20, 60, 100으로 나누어 표현했다. 이렇게 진행하면 1~20, 21~60, 61~100으로 구분하는 것과 같다.

> 별도의 이름을 지정하기 위해 age_labels를 만들었다.(나는 만들지않고 그냥썼다)

> 이렇게 만든 것을 age_class라는 feature에 저장했다. 

In [6]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
Age_group       14
dtype: int64

> age_group에서 결측치가 14개 발견되었다. 이는 그룹을 1살부터 시작했기 때문인것으로 보인다.

In [7]:
df[df['Age']<1].describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,7.000000,7.0,7.000000,7.000000,7.000000,7.000000,7.000000
mean,556.000000,1.0,2.285714,0.738571,1.000000,1.285714,37.261900
std,283.732856,0.0,0.755929,0.161289,0.816497,0.487950,50.769691
min,79.000000,1.0,1.000000,0.420000,0.000000,1.000000,8.516700
25%,388.000000,1.0,2.000000,0.710000,0.500000,1.000000,16.625000
50%,645.000000,1.0,2.000000,0.750000,1.000000,1.000000,19.258300
75%,780.000000,1.0,3.000000,0.830000,1.500000,1.500000,24.129150
max,832.000000,1.0,3.000000,0.920000,2.000000,2.000000,151.550000


> 실행결과 1살 미만의 아이는 7명이 나왔다.

In [8]:
df[df['Age']<=1].describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000
mean,485.714286,0.857143,2.500000,0.869286,1.428571,1.428571,33.633929
std,282.697019,0.363137,0.650444,0.174376,1.504572,0.513553,35.956424
min,79.000000,0.000000,1.000000,0.420000,0.000000,1.000000,8.516700
25%,214.500000,1.000000,2.000000,0.770000,0.250000,1.000000,16.493775
50%,428.500000,1.000000,3.000000,0.960000,1.000000,1.000000,19.916650
75%,780.750000,1.000000,3.000000,1.000000,2.000000,2.000000,38.501050
max,832.000000,1.000000,3.000000,1.000000,5.000000,2.000000,151.550000


> 1을 포함한 개수를 확인해 보니 14개가 나왔다.

> 결국 cut함수에서 age_bins는 첫 1의 의미가 1 이상이 아닌 1초과로 처리되는 것을 알 수 있다. == 1부터 시작해서 그룹핑하나 1을 포함하지 않는다. (1~n]

> 그래서 결국 범위를 0부터 시작해줘야 한다.

In [9]:
df['Age_group'] = pd.cut(df['Age'], bins=[0, 20 ,60, 100], labels=['미성년', '성년', '노년'])
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_group
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S,성년
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,성년
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S,성년
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,성년
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S,성년
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,NaN,S,성년
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,B42,S,미성년
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.699118,1,2,W./C. 6607,23.4500,NaN,S,성년
889,890,1,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.0000,C148,C,성년


In [10]:
# 범위를 0부터 주니, Age_group의 결측치가 사라사라사라져따리다
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
Age_group        0
dtype: int64

##3. 데이터 필터링
> 데이터 가공 시 중복 데이터를 제거한다거나 필요한 데이터만 분류하는 경우가 자주 발생된다. 여기서는 이러한 필터링 방법에 대해 알아보자

###3.1 중복 제거

In [11]:
#df['Age_group']
df['Age_group'].unique()

['성년', '미성년', '노년']
Categories (3, object): ['미성년' < '성년' < '노년']

###3.2 데이터 분류

In [12]:
df[df['Age_group'].isin(['미성년'])]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_group
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,미성년
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,미성년
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S,미성년
12,13,0,3,"Saundercock, Mr. William Henry",male,20.0,0,0,A/5. 2151,8.0500,NaN,S,미성년
14,15,0,3,"Vestrom, Miss. Hulda Amanda Adolfina",female,14.0,0,0,350406,7.8542,NaN,S,미성년
...,...,...,...,...,...,...,...,...,...,...,...,...,...
869,870,1,3,"Johnson, Master. Harold Theodor",male,4.0,1,1,347742,11.1333,NaN,S,미성년
875,876,1,3,"Najib, Miss. Adele Kiamie ""Jane""",female,15.0,0,0,2667,7.2250,NaN,C,미성년
876,877,0,3,"Gustafsson, Mr. Alfred Ossian",male,20.0,0,0,7534,9.8458,NaN,S,미성년
877,878,0,3,"Petroff, Mr. Nedelio",male,19.0,0,0,349212,7.8958,NaN,S,미성년


> isin함수는 인자로 전달되는 리스트 정보가 포함된 데이터를 분류한다.

> 위의 코드는 Age_group중 미성년인 정보만 추출하는 코드이다.

In [13]:
df[~df['Age_group'].isin(['미성년'])]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_group
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S,성년
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,성년
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S,성년
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,성년
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S,성년
...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.000000,0,5,382652,29.1250,NaN,Q,성년
886,887,0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,NaN,S,성년
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.699118,1,2,W./C. 6607,23.4500,NaN,S,성년
889,890,1,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.0000,C148,C,성년


> 보통 프로그램에서 틸트(~)는 not의 의미를 갖게 된다. 위의 코드에서 ~를 적용하였기 때문에 미성년이 배제되어 있는 모든 데이터를 추출해라 라는 의미로 동작된다.

##4. 생존률 분석

###4.1 unstack
> 성별, 연령대별, 객실 등급별 생존률 확인

In [14]:
df_survived = df.groupby(['Sex', 'Age_group', 'Pclass'])['Survived'].mean()
df_survived

Sex     Age_group  Pclass
female  미성년        1         0.928571
                   2         1.000000
                   3         0.510638
        성년         1         0.974359
                   2         0.900000
                   3         0.489583
        노년         1         1.000000
                   2              NaN
                   3         1.000000
male    미성년        1         0.571429
                   2         0.526316
                   3         0.197368
        성년         1         0.388350
                   2         0.069767
                   3         0.119850
        노년         1         0.083333
                   2         0.333333
                   3         0.000000
Name: Survived, dtype: float64

> 성별 기준으로 그룹한 후 연령대별, 객실등급별 순으로 그룹화 한 후 생존데이터들에 대한 평균을 구하여 생존률을 얻었다. 하지만 보기가 어렵다. 이를 해결하기 위해 다음과 같이 unstack함수를 이용한다.

In [15]:
df_survived.unstack('Pclass')

Pclass                   1         2         3
Sex    Age_group                              
female 미성년        0.928571  1.000000  0.510638
       성년         0.974359  0.900000  0.489583
       노년         1.000000       NaN  1.000000
male   미성년        0.571429  0.526316  0.197368
       성년         0.388350  0.069767  0.119850
       노년         0.083333  0.333333  0.000000

> unstack은 열을 행으로 변환하는 기능을 한다. 이전 코드에서 Pclass가 열로 되어있는 부분을 이번 코드에서 행으로 변환하여 출력했다.

###4.2 pivot_table

> 성별에 따른 생존율 확인

In [16]:
df.pivot_table(['Survived'], index=['Sex'])

,Survived
Sex,
female,0.742038
male,0.188908


> pivot_table은 행과 열을 입력받아 새로운 데이터를 구성하는 함수이다. 위의 실행 결과 여성의 생존률이 높은 것을 알 수있다.

In [17]:
df.pivot_table(['Survived'], index=['Sex', 'Pclass'])

Survived
Sex    Pclass          
female 1       0.968085
       2       0.921053
       3       0.500000
male   1       0.368852
       2       0.157407
       3       0.135447

##5. 자동 보고서 만들기
> colab에서는 기본적으로 pandas-profiling이 설치되어 있지만 버전이 낮아 profile_report()함수를 사용할 수 없다.

> 다음과 같이 pip를 이용하여 업데이트 한후 report를 확인해 보자.

In [18]:
!pip install -U pandas-profiling

Requirement already up-to-date: pandas-profiling in /usr/local/lib/python3.7/dist-packages (3.0.0)


> 위의 명령어 실행 이후 메뉴에서 런타임 -> 런타임 다시시작을 진행해야 한다.

In [21]:
import pandas_profiling
report = df.profile_report()
url = '/content/drive/MyDrive/Colab Notebooks/Lec4_BigData/3. 파이프라인/titanic_report.html'

report.to_file(url)

Summarize dataset:   0%|          | 0/26 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/matplotlib/textpath.py:84: RuntimeWarning: Glyph 49457 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/usr/local/lib/python3.7/dist-packages/matplotlib/textpath.py:84: RuntimeWarning: Glyph 45380 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/usr/local/lib/python3.7/dist-packages/matplotlib/textpath.py:84: RuntimeWarning: Glyph 48120 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/usr/local/lib/python3.7/dist-packages/matplotlib/textpath.py:84: RuntimeWarning: Glyph 45432 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/usr/local/lib/python3.7/dist-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 49457 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/usr/local/lib/python3.7/dist-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 45380 missing from current font.
  font.set_text(s, 0.0,

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.002015,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200
